In [ ]:
# if hit plotting library issues, try resetting LD path for julia
# can set in ~/.local/share/jupyter/kernels/
haskey(ENV, "LD_LIBRARY_PATH") && @assert ENV["LD_LIBRARY_PATH"] == ""

import Pkg
# use temp or named environment to avoid package clashes across development projects
Pkg.activate(;temp=true)
Pkg.add("Revise")
import Revise

Pkg.develop(path="$(homedir())/workspace/Mycelia")
import Mycelia

pkgs = [
    "DataFrames",
    "CSV",
    "uCSV"
]
Pkg.add(pkgs)
for pkg in pkgs
    eval(Meta.parse("import $pkg"))
end

In [ ]:
PROJECT_BASEDIR = dirname(pwd())
data_dir = joinpath(PROJECT_BASEDIR, "data")
genome_dir = mkpath(joinpath(data_dir, "genomes"))

In [ ]:
readdir(data_dir)

In [ ]:
VMR_MSL_file = joinpath(data_dir, "VMR_MSL39_v1.tsv")
VMR_MSL_table = DataFrames.DataFrame(uCSV.read(VMR_MSL_file, delim='\t', header=1))
is_examplar = VMR_MSL_table[!, "Exemplar or additional isolate"] .== "E"
in_refseq = .!isempty.(VMR_MSL_table[!, "Virus REFSEQ accession"]) .& .!occursin.(";", VMR_MSL_table[!, "Virus REFSEQ accession"])
refseq_exemplar_table = VMR_MSL_table[is_examplar .& in_refseq, :]

In [ ]:
# 7
# unique(exemplar_table[!, "Realm"])

# 11
# unique(exemplar_table[!, "Kingdom"])

# 19
# unique(exemplar_table[!, "Phylum"])

one_per_realm_df = DataFrames.DataFrame()
for gdf in DataFrames.groupby(refseq_exemplar_table, "Realm")
    @assert issorted(gdf, "Sort")
    push!(one_per_realm_df, gdf[1, :])
end
one_per_realm_df

In [ ]:
for row in DataFrames.eachrow(one_per_realm_df)
    outfile = Mycelia.download_genome_by_accession(accession = row["Virus REFSEQ accession"], outdir = genome_dir, compressed=false)
end